#### Hypertuning of ridge regression with momentum

In [ ]:
include("../code/optimization.jl")
include("../code/franke.jl")
include("../code/hypertuning.jl")
include("../code/utils.jl")

using DataFrames
using Random

Random.seed!(0)

n_samples = 1000
noise_amplitude = 0.01
random_inputs = false

X, y = franke_training_data(n_samples, noise_amplitude, random_inputs)
X_scaled = standardize_data(X)

k_folds = 10
trials = 1000
max_degrees = 10
lr_range = (1e-4, 0.1)
l2_lambda_range = (1e-6, 0.1)
momentum_range = (0.0, 0.9)

best_idx, results = ridge_momentum_gradient_descent_random_search(
  X, y,
  k_folds,
  trials,
  max_degrees,
  lr_range,
  l2_lambda_range,
  momentum_range,
)

df = DataFrame(
  degree=[r.degree for r in results],
  learning_rate=[r.learning_rate for r in results],
  l2_lambda=[r.l2_lambda for r in results],
  momentum=[r.momentum for r in results],
  mse=[r.mse_mean_std[1] for r in results],
  mse_std=[r.mse_mean_std[2] for r in results],
  r2=[r.r2_mean_std[1] for r in results],
  r2_std=[r.r2_mean_std[2] for r in results],
)

headers = OrderedDict{String,Union{Symbol,Tuple{Symbol,Symbol}}}(
  "Degree" => :degree,
  "\$\\eta\$" => :learning_rate,
  "\$\\lambda\$" => :l2_lambda,
  "\$\\gamma\$" => :momentum,
  "MSE" => (:mse, :mse_std),
  "R^2" => (:r2, :r2_std),
)

df = sort(df, :mse)
markdown_table = dataframe_to_markdown_table(first(df, 10), headers)
open("ridge_momentum_hypertuning.txt", "w") do io
  write(io, markdown_table)
end

1642

1642